- trade futures 
- retreat strategy, each clip higher credit to avoid position 
    - people usually get stuck in their positions
    - markets get extremely liquid
- instead of mid point use bid and ask of underlying spread
- Paper:
    - price based on latest trading price
    - adjust ask/bid spread based on market volatility
    - trend follow strategy driven by order imbalance

In [ ]:
import datetime as dt
import time
import logging

from optibook.synchronous_client import Exchange
from optibook.common_types import InstrumentType, OptionKind

from math import floor, ceil, exp
from black_scholes import call_value, put_value, call_delta, put_delta
from libs import calculate_current_time_to_date

exchange = Exchange()
exchange.connect()

logging.getLogger('client').setLevel('ERROR')

In [2]:
def trade_would_breach_position_limit(instrument_id, volume, side, position_limit=100):
    positions = exchange.get_positions()
    position_instrument = positions[instrument_id]

    if side == 'bid':
        return position_instrument + volume > position_limit
    elif side == 'ask':
        return position_instrument - volume < -position_limit
    else:
        raise Exception(f'''Invalid side provided: {side}, expecting 'bid' or 'ask'.''')


In [3]:
def round_down_to_tick(price, tick_size):
    """
    Rounds a price down to the nearest tick, e.g. if the tick size is 0.10, a price of 0.97 will get rounded to 0.90.
    """
    return floor(price / tick_size) * tick_size


def round_up_to_tick(price, tick_size):
    """
    Rounds a price up to the nearest tick, e.g. if the tick size is 0.10, a price of 1.34 will get rounded to 1.40.
    """
    return ceil(price / tick_size) * tick_size



In [4]:
def load_futures_for_underlying(underlying_group_id):

    all_instruments = exchange.get_instruments()

    futures = {instrument_id: instrument
               for instrument_id, instrument in all_instruments.items()
               if (instrument.instrument_type == InstrumentType.INDEX_FUTURE or instrument.instrument_type == InstrumentType.STOCK_FUTURE)
               and instrument.instrument_group == underlying_group_id}
    
    return futures

In [5]:
def load_instruments_for_underlying(underlying_stock_id):
    all_instruments = exchange.get_instruments()
    stock = all_instruments[underlying_stock_id]
    options = {instrument_id: instrument
               for instrument_id, instrument in all_instruments.items()
               if instrument.instrument_type == InstrumentType.STOCK_OPTION
               and instrument.base_instrument_id == underlying_stock_id}
    return options

In [6]:
def load_instruments_for_OB5X():
    all_instruments = exchange.get_instruments()
    filtered_dict = {key: value for key, value in all_instruments.items() if value.instrument_group == 'OB5X' and value.instrument_type == InstrumentType.INDEX_OPTION}
    return filtered_dict

In [7]:
def get_midpoint_value(instrument_id):
    """
    This function calculates the current midpoint of the order book supplied by the exchange for the instrument
    specified by <instrument_id>, returning None if either side or both sides do not have any orders available.
    """
    order_book = exchange.get_last_price_book(instrument_id=instrument_id)

    # If the instrument doesn't have prices at all or on either side, we cannot calculate a midpoint and return None
    if not (order_book and order_book.bids and order_book.asks):
        return None
    else:
        midpoint = (order_book.bids[0].price + order_book.asks[0].price) / 2.0
        return midpoint

In [8]:
def get_OB5X_value(side):
    """
    Calculates the value of the OB5X index
    """
    
    val = 0 
    if(side=='ask'):
        if(len(exchange.get_last_price_book('NVDA').asks)==0 or len(exchange.get_last_price_book('ING').asks)==0 or len(exchange.get_last_price_book('SAN').asks)==0 or len(exchange.get_last_price_book('PFE').asks)==0 or len(exchange.get_last_price_book('CSCO').asks) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').asks[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').asks[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').asks[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').asks[0].price
        CSCO = exchange.get_last_price_book(instrument_id='CSCO').asks[0].price

        val = (908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE + 953.21*CSCO)/1000
    else:
        if(len(exchange.get_last_price_book('NVDA').bids)==0 or len(exchange.get_last_price_book('ING').bids)==0 or len(exchange.get_last_price_book('SAN').bids)==0 or len(exchange.get_last_price_book('PFE').bids)==0 or len(exchange.get_last_price_book('CSCO').bids) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').bids[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').bids[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').bids[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').bids[0].price
        CSCO = exchange.get_last_price_book(instrument_id='CSCO').bids[0].price

        val = (908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE + 953.21*CSCO)/1000
    

    return val

In [9]:
def ETF_val(index_val):
    C = 2.5
    M = 0.25
    return C + M*index_val

In [10]:
def calculate_theoretical_future_value(underlying_value, expiry,  interest_rate=0.03):
    """
    This function calculates the current fair future value with: F = X exp(rt)

    expiry: dt.date          -  Expiry date of the future -  third Friday of current month, at 12:00 UTC
    underlying_value:        -  value of the underlying stock/index
    interest_rate:           -  interest rate of future
    """
    time_to_expiry = calculate_current_time_to_date(expiry)

    future_value = underlying_value*exp(interest_rate*time_to_expiry)

    return future_value

In [11]:
def calculate_theoretical_option_value(expiry, strike, option_kind, stock_value, interest_rate, volatility):
    """
    This function calculates the current fair call or put value based on Black & Scholes assumptions.

    expiry: dt.date          -  Expiry date of the option
    strike: float            -  Strike price of the option
    option_kind: OptionKind  -  Type of the option
    stock_value:             -  Assumed stock value when calculating the Black-Scholes value
    interest_rate:           -  Assumed interest rate when calculating the Black-Scholes value
    volatility:              -  Assumed volatility of when calculating the Black-Scholes value
    """
    time_to_expiry = calculate_current_time_to_date(expiry)


    if option_kind == OptionKind.CALL:
        option_value = call_value(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)
    elif option_kind == OptionKind.PUT:
        option_value = put_value(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)

    return option_value

In [12]:
def update_quotes(instrument_id, theoretical_price, credit, volume, position_limit, tick_size):
    """
    This function updates the quotes specified by <option_id>. We take the following actions in sequence:
        - pull (remove) any current oustanding orders
        - add credit to theoretical price and round to nearest tick size to create a set of bid/ask quotes
        - calculate max volumes to insert as to not pass the position_limit
        - reinsert limit orders on those levels

    Arguments:
        option_id: str           -  Exchange Instrument ID of the option to trade
        theoretical_price: float -  Price to quote around
        credit: float            -  Difference to subtract from/add to theoretical price to come to final bid/ask price
        volume:                  -  Volume (# lots) of the inserted orders (given they do not breach position limits)
        position_limit: int      -  Position limit (long/short) to avoid crossing
        tick_size: float         -  Tick size of the quoted instrument
    """

    # Print any new trades
    trades = exchange.poll_new_trades(instrument_id=instrument_id)

    # Pull (remove) all existing outstanding orders
    orders = exchange.get_outstanding_orders(instrument_id=instrument_id)
    for order_id, order in orders.items():
        exchange.delete_order(instrument_id=instrument_id, order_id=order_id)

    # Calculate bid and ask price
    bid_price = round_down_to_tick(theoretical_price - credit, tick_size)
    ask_price = round_up_to_tick(theoretical_price + credit, tick_size)

    # Calculate bid and ask volumes, taking into account the provided position_limit
    position = exchange.get_positions()[instrument_id]

    max_volume_to_buy = position_limit - position
    max_volume_to_sell = position_limit + position

    bid_volume = min(volume, max_volume_to_buy)
    ask_volume = min(volume, max_volume_to_sell)

    # Insert new limit orders
    if bid_volume > 0:
        exchange.insert_order(
            instrument_id=instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        print(f'- Inserting bid limit order in {instrument_id} for {bid_volume} @ {bid_price:8.2f}.')
    if ask_volume > 0:
        exchange.insert_order(
            instrument_id=instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )
        print(f'- Inserting ask limit order in {instrument_id} for {ask_volume} @ {ask_price:8.2f}.')

In [13]:
def calculate_option_delta(expiry_date, strike, option_kind, stock_value, interest_rate, volatility):
    """
    This function calculates the current option delta based on Black & Scholes assumptions.

    expiry_date: dt.date     -  Expiry date of the option
    strike: float            -  Strike price of the option
    option_kind: OptionKind  -  Type of the option
    stock_value:             -  Assumed stock value when calculating the Black-Scholes value
    interest_rate:           -  Assumed interest rate when calculating the Black-Scholes value
    volatility:              -  Assumed volatility of when calculating the Black-Scholes value
    """
    time_to_expiry = calculate_current_time_to_date(expiry_date)

    if option_kind == OptionKind.CALL:
        option_delta = call_delta(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)
    elif option_kind == OptionKind.PUT:
        option_delta = put_delta(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)
    else:
        raise Exception(f"""Got unexpected value for option_kind argument, should be OptionKind.CALL or OptionKind.PUT but was {option_kind}.""")

    return option_delta


In [14]:
def hedge_delta_position(stock_id, stock_value, futures=None, options=None, dual=None):
    """
    This function (once finished) hedges the outstanding delta position by trading in the stock.

    That is:
        - It calculates how sensitive the total position value is to changes in the underlying by summing up all
          individual delta component.
        - And then trades stocks which have the opposite exposure, to remain, roughly, flat delta exposure

    Arguments:
        stock_id: str         -  Exchange Instrument ID of the stock to hedge with
        options: List[dict]   -  List of options with details to calculate and sum up delta positions for
        stock_value: float    -  The stock value to assume when making delta calculations using Black-Scholes
    """

    # A2: Calculate the delta position here
    positions = exchange.get_positions()

    total_delta = 0

    # hedge options
    if options != None:
        for option_id, option in options.items():
            position = positions[option_id]
            current_delta = calculate_option_delta(option.expiry, option.strike, option.option_kind, stock_value, 0.03, 3.0)
            total_delta += current_delta * position
            print(f"- The current position in option {option_id} is {position}.")

    # hedge futures
    if futures != None:
        for future_id, future in futures.items():
            position = positions[future_id]
            # current_delta = calculate_future_delta()
            total_delta += position

    # hedge dual 
    if dual != None:
        total_delta += positions[dual]


    stock_position = positions[stock_id]

    stock_order_book = exchange.get_last_price_book(stock_id)

    # if empty hedge order book escape function
    if not (stock_order_book and stock_order_book.bids and stock_order_book.asks):
            return

    # compare position in underlying stock to necessary position for hedging and determine trade volume
    trade_volume = 0
    if total_delta != -stock_position:
        if total_delta < 0 and stock_position > 0:
            trade_volume = abs(stock_position + round(total_delta))
        elif total_delta > 0 and stock_position < 0:
            trade_volume = abs(stock_position + round(total_delta))
        else:
            trade_volume = abs(round(total_delta) - stock_position)
    else:
        return
    
    # instert orders into hedge instrument order book
    if trade_volume != 0:
        if stock_position < total_delta:
            if abs(stock_position) < total_delta:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'ask'):
                    exchange.insert_order(stock_id, price = 1, volume=trade_volume, side='ask', order_type='ioc')
            else:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'bid'):
                    exchange.insert_order(stock_id, price = 10000, volume=trade_volume, side='bid', order_type='ioc')
        elif total_delta < stock_position:
            if abs(total_delta) < stock_position:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'ask'):
                    exchange.insert_order(stock_id, price = 1, volume=trade_volume, side='ask', order_type='ioc')
            else:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'bid'):
                    exchange.insert_order(stock_id, price = 10000, volume=trade_volume, side='bid', order_type='ioc')
                    
    # if delta is more than we can hedge
    positions = exchange.get_positions()
    stock_position = positions[stock_id]
    print(f"stock position currently: {stock_position}")
    if stock_position != 100 and stock_position != -100:
        if total_delta >= 100:
            exchange.insert_order(stock_id, price = 1, volume = 100 + stock_position, side = 'ask', order_type = 'ioc')
        elif total_delta <= -100:
            print(f"stock position currently: {positions[stock_id]}")
            exchange.insert_order(stock_id, price = 10000, volume = 100 - stock_position, side = 'bid', order_type = 'ioc')

In [15]:
def trade_out_of_position():
    MIN_SELLING_PRICE = 0.10
    MAX_BUYING_PRICE = 100000.00

    positions = exchange.get_positions()
    for iid, pos in positions.items():
        if pos > 0:
            exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
        elif pos < 0:
            exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')    
        time.sleep(0.10)

trade_out_of_position()

In [ ]:
def hedge_delta_position(stock_id, stock_value, futures=None, options=None, dual=None):
    """
    This function (once finished) hedges the outstanding delta position by trading in the stock.

    That is:
        - It calculates how sensitive the total position value is to changes in the underlying by summing up all
          individual delta component.
        - And then trades stocks which have the opposite exposure, to remain, roughly, flat delta exposure

    Arguments:
        stock_id: str         -  Exchange Instrument ID of the stock to hedge with
        options: List[dict]   -  List of options with details to calculate and sum up delta positions for
        stock_value: float    -  The stock value to assume when making delta calculations using Black-Scholes
    """

    # A2: Calculate the delta position here
    positions = exchange.get_positions()

    total_delta = 0

    # hedge options
    if options != None:
        if stock_id == 'OB5X_ETF':
            for option_id, option in options.items():
                position = positions[option_id]
                current_delta = calculate_option_delta(option.expiry, option.strike, option.option_kind, stock_value, 0.03, 3.0)
                current_delta = current_delta / 0.25
                total_delta += current_delta * position
                print(f"- The current position in option {option_id} is {position}.")
        else:
            for option_id, option in options.items():
                position = positions[option_id]
                current_delta = calculate_option_delta(option.expiry, option.strike, option.option_kind, stock_value, 0.03, 3.0)
                total_delta += current_delta * position
                print(f"- The current position in option {option_id} is {position}.")

    # hedge futures
    if futures != None:
        if stock_id == 'OB5X_ETF':
            for future_id, future in futures.items():
                position = positions[future_id]
                current_delta = exp(0.03*calculate_current_time_to_date(future.expiry))/0.25
                total_delta += position * current_delta
        elif stock_id == 'NVDA':
            for future_id, future in futures.items():
                position = positions[future_id]
                total_delta += current_delta

    # hedge dual 
    if dual != None:
        total_delta += positions[dual]


    stock_position = positions[stock_id]

    stock_order_book = exchange.get_last_price_book(stock_id)

    # if empty hedge order book escape function
    if not (stock_order_book and stock_order_book.bids and stock_order_book.asks):
            return

    # compare position in underlying stock to necessary position for hedging and determine trade volume
    trade_volume = 0
    if total_delta != -stock_position:
        if total_delta < 0 and stock_position > 0:
            trade_volume = abs(stock_position + round(total_delta))
        elif total_delta > 0 and stock_position < 0:
            trade_volume = abs(stock_position + round(total_delta))
        else:
            trade_volume = abs(round(total_delta) - stock_position)
    else:
        return
    
    # instert orders into hedge instrument order book
    if trade_volume != 0:
        if stock_position < total_delta:
            if abs(stock_position) < total_delta:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'ask'):
                    exchange.insert_order(stock_id, price = 1, volume=trade_volume, side='ask', order_type='ioc')
            else:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'bid'):
                    exchange.insert_order(stock_id, price = 10000, volume=trade_volume, side='bid', order_type='ioc')
        elif total_delta < stock_position:
            if abs(total_delta) < stock_position:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'ask'):
                    exchange.insert_order(stock_id, price = 1, volume=trade_volume, side='ask', order_type='ioc')
            else:
                if not trade_would_breach_position_limit(stock_id, trade_volume, 'bid'):
                    exchange.insert_order(stock_id, price = 10000, volume=trade_volume, side='bid', order_type='ioc')
                    
    # if delta is more than we can hedge
    positions = exchange.get_positions()
    stock_position = positions[stock_id]
    print(f"stock position currently: {stock_position}")
    if stock_position != 100 and stock_position != -100:
        if total_delta >= 100:
            exchange.insert_order(stock_id, price = 1, volume = 100 + stock_position, side = 'ask', order_type = 'ioc')
        elif total_delta <= -100:
            print(f"stock position currently: {positions[stock_id]}")
            exchange.insert_order(stock_id, price = 10000, volume = 100 - stock_position, side = 'bid', order_type = 'ioc')

In [34]:
def credit_calc (instrument_id, volume_steps, increment):
    position = exchange.get_positions()
    instrument_position = position[instrument_id]
    credit = -round(instrument_position / volume_steps) * increment
    return credit

In [47]:
NVDA_ID = "NVDA"
NVDA_DUAL = "NVDA_DUAL"
OB5X_ID = "OB5X"
OB5X_ETF = 'OB5X_ETF'

flag = True
while True:
    OB5X_value_ask = get_OB5X_value('ask')
    OB5X_value_bid = get_OB5X_value('bid')
    options = load_instruments_for_OB5X()
    order_book = exchange.get_last_price_book(OB5X_ETF)

    if (OB5X_value_ask is None or OB5X_value_bid is None):
        continue

    for option_id, option in options.items():
        if (len(order_book.bids) == 0 or len(order_book.asks) == 0):
            continue

        theoretical_value_bid = calculate_theoretical_option_value(expiry = option.expiry,
                                                                   strike = option.strike,
                                                                   option_kind = option.option_kind,
                                                                   stock_value = OB5X_value_bid,
                                                                   interest_rate = 0.03,
                                                                   volatility = 1.5)
        position = exchange.get_last_price_book(option_id)

        option_price = position.bids[0].price
        option_price1 = position.asks[0].price
        
        
        
        print(f"option id: {option_id}")
        print(f"theoretical bid: {theoretical_value_bid}")
        print(f"best bid: {option_price}")
        
        theoretical_value_ask = calculate_theoretical_option_value(expiry = option.expiry,
                                                                   strike = option.strike,
                                                                   option_kind = option.option_kind,
                                                                   stock_value = OB5X_value_ask,
                                                                   interest_rate = 0.03,
                                                                   volatility = 1.5)
        print(f"theoretical ask: {theoretical_value_ask}")
        print(f"best ask: {option_price1}")

        
        if option.option_kind == OptionKind.CALL:
            price_diff = (theoretical_value_ask - theoretical_value_bid)
            mid_price = theoretical_value_bid + price_diff/2

        elif option.option_kind == OptionKind.PUT:
            price_diff = (theoretical_value_bid - theoretical_value_ask)
            mid_price = theoretical_value_ask + price_diff/2

        update_quotes(instrument_id=option_id,
                        theoretical_price = mid_price + credit_calc(option_id, 10, 0.1),
                        credit=price_diff/2 + 0.1,
                        volume=33,
                        position_limit=100,
                        tick_size=0.10)
        
        for i in range(1):
            hedge_delta_position(OB5X_ETF, get_midpoint_value(OB5X_ETF), futures=None, options=options, dual=None)
            time.sleep(1)
    

option id: OB5X_202406_120P
theoretical bid: 35.92777373938556
best bid: 35.5
theoretical ask: 35.74795213837395
best ask: 36.0
- Inserting bid limit order in OB5X_202406_120P for 33 @    35.50.
- Inserting ask limit order in OB5X_202406_120P for 33 @    36.00.
- The current position in option OB5X_202406_120P is 12.
- The current position in option OB5X_202406_120C is 2.
- The current position in option OB5X_202406_100P is 3.
- The current position in option OB5X_202406_100C is 20.
- The current position in option OB5X_202406_080C is 4.
- The current position in option OB5X_202406_080P is -2.
stock position currently: -48
option id: OB5X_202406_120C
theoretical bid: 56.63128092849935
best bid: 56.6
theoretical ask: 57.13746832497861
best ask: 56.800000000000004
- Inserting bid limit order in OB5X_202406_120C for 33 @    56.50.
- Inserting ask limit order in OB5X_202406_120C for 33 @    57.30.
- The current position in option OB5X_202406_120P is 12.
- The current position in option OB5

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
NVDA_ID = "NVDA"
NVDA_DUAL = "NVDA_DUAL"
OB5X_ID = "OB5X"
OB5X_ETF = 'OB5X_ETF'

flag = True
i = 0
while True:
    OB5X_value_ask = get_OB5X_value('ask')
    OB5X_value_bid = get_OB5X_value('bid')
    futures = load_futures_for_underlying('OB5X')
    if (OB5X_value_ask is None or OB5X_value_bid is None):
        time.sleep(1)
        continue
    for future_id, future in futures.items():
        theoretical_value_bid = calculate_theoretical_future_value(underlying_value = OB5X_value_bid, expiry = future.expiry)
        theoretical_value_ask = calculate_theoretical_future_value(underlying_value = OB5X_value_ask, expiry = future.expiry)
        diff = theoretical_value_ask - theoretical_value_bid
        mid_price = theoretical_value_bid + diff/2
        
        #updating future quotes

        update_quotes(instrument_id = future_id,
                      theoretical_price = mid_price + credit_calc(future_id, 20, 0.1),
                      credit = diff/2 + 0.2,
                      volume = 20,
                      position_limit = 100,
                      tick_size = 0.10)
        
        time.sleep(1)
    
        for i in range(1):
            hedge_delta_position(OB5X_ETF, get_midpoint_value(OB5X_ETF), futures=futures, options=None, dual=None)
            time.sleep(1)


    # ---------------------------------------------------- NVDA --------------------------------------------------
    # NVDA options
    """
    options = load_instruments_for_underlying(NVDA_ID)
    futures = load_futures_for_underlying(NVDA_ID)
    for option_id, option in options.items():
        order_book = exchange.get_last_price_book(NVDA_ID)

        # check if order book is empty
        if(len(order_book.bids)==0 or len(order_book.asks)==0):
            continue

        theoretical_value_bid = calculate_theoretical_option_value(expiry=option.expiry,
                                                               strike=option.strike,
                                                               option_kind=option.option_kind,
                                                               stock_value=order_book.bids[0].price,
                                                               interest_rate=0.03,
                                                               volatility=3.0)

        theoretical_value_ask = calculate_theoretical_option_value(expiry=option.expiry,
                                                               strike=option.strike,
                                                               option_kind=option.option_kind,
                                                               stock_value=order_book.asks[0].price,
                                                               interest_rate=0.03,
                                                               volatility=3.0)

        if option.option_kind == OptionKind.CALL:
            price_diff = (theoretical_value_ask - theoretical_value_bid)
            mid_price = theoretical_value_bid + price_diff/2

        elif option.option_kind == OptionKind.PUT:
            price_diff = (theoretical_value_bid - theoretical_value_ask)
            mid_price = theoretical_value_ask + price_diff/2

        update_quotes(instrument_id=option_id,
                        theoretical_price = mid_price,
                        credit=price_diff/2+ 0.1,
                        volume=50,
                        position_limit=100,
                        tick_size=0.10)
    time.sleep(1.0)
    
    # hedge all NVDA positions
    hedge_delta_position(NVDA_ID, get_midpoint_value(NVDA_ID), futures=futures, options=options, dual=NVDA_DUAL)

    # NVDA futures
    
    for future_id,future in futures.items():
        order_book = exchange.get_last_price_book(NVDA_ID)

        # check if order book is empty
        if(len(order_book.bids)==0 or len(order_book.asks)==0):
            continue
        theoretical_value_bid = calculate_theoretical_future_value(underlying_value=order_book.bids[0].price, expiry=future.expiry)
        theoretical_value_ask = calculate_theoretical_future_value(underlying_value=order_book.asks[0].price, expiry=future.expiry)
        diff = theoretical_value_ask-theoretical_value_bid
        mid_price = theoretical_value_bid+diff/2

        update_quotes(instrument_id=future_id,
                        theoretical_price = mid_price,
                        credit=diff/2 + 0.1,
                        volume=50,
                        position_limit=100,
                        tick_size=0.10)

    # NVDA_DUAL
    order_book = exchange.get_last_price_book(NVDA_ID)
    bid_price = order_book.bids[0].price
    ask_price = order_book.asks[0].price

    mid_point = (ask_price + bid_price)/2
    creditdual = (ask_price - bid_price)/2 + 0.20
    update_quotes(instrument_id=NVDA_DUAL,
                theoretical_price = mid_point,
                credit = creditdual,
                volume = 20,
                position_limit = 100,
                tick_size = 0.10)
    time.sleep(1.0)

    hedge_delta_position(NVDA_ID, get_midpoint_value(NVDA_ID), futures=futures, options=options, dual=NVDA_DUAL)
      """
    



    

Instrument(instrument_id=OB5X_202412_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_FUTURE, expiry=2024-12-20 12:00:00, interest_rate=0.03, index_id=OB5X, instrument_group=OB5X, paused=False, expired=False)
140.374979
139.900139
theoretical value bid at: 143.61309210393406
OB5X_202412_F best bid at: 143.3
theoretical value ask at: 144.1005343691149
OB5X_202412_F best ask at: 143.8
- Inserting bid limit order in OB5X_202412_F for 20 @   143.40.
- Inserting ask limit order in OB5X_202412_F for 20 @   144.40.
stock position currently: 20
Instrument(instrument_id=OB5X_202409_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_FUTURE, expiry=2024-09-20 12:00:00, interest_rate=0.03, index_id=OB5X, instrument_group=OB5X, paused=False, expired=False)
140.374979
139.900139
theoretical value bid at: 142.54295160794283

IndexError: list index out of range

In [32]:
OB5X_ID = 'OB5X'

futures = load_futures_for_underlying(OB5X_ID)
print(futures)

for future in futures:
    print(calculate_theoretical_future_value(get_OB5X_value('ask'), futures[future].expiry))
    print(calculate_theoretical_future_value(get_OB5X_value('bid'), futures[future].expiry))

print(exchange.get_last_price_book('OB5X_202412_F'))
print(exchange.get_last_price_book('OB5X_202406_F'))
print(exchange.get_last_price_book('OB5X_202409_F'))

{'OB5X_202412_F': Instrument(instrument_id=OB5X_202412_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_FUTURE, expiry=2024-12-20 12:00:00, interest_rate=0.03, index_id=OB5X, instrument_group=OB5X, paused=False, expired=False), 'OB5X_202409_F': Instrument(instrument_id=OB5X_202409_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_FUTURE, expiry=2024-09-20 12:00:00, interest_rate=0.03, index_id=OB5X, instrument_group=OB5X, paused=False, expired=False), 'OB5X_202406_F': Instrument(instrument_id=OB5X_202406_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_FUTURE, expiry=2024-06-21 12:00:00, interest_rate=0.03, index_id=OB5X, instrument_group=OB5X, paused=False, expired=False)}
145.7650937668278
145.22423589292072
144.6789

In [18]:
len(order_book.bids)==0

False

In [49]:
for future_id,future in futures.items():
    print(exchange.get_outstanding_orders(future_id))
    

{93717909: OrderStatus(order_id=93717909, instrument_id=NVDA_202412_F, price=75.0, volume=100, side=bid), 93717910: OrderStatus(order_id=93717910, instrument_id=NVDA_202412_F, price=75.5, volume=50, side=ask)}
{93717930: OrderStatus(order_id=93717930, instrument_id=NVDA_202406_F, price=73.9, volume=100, side=bid)}
{93717940: OrderStatus(order_id=93717940, instrument_id=NVDA_202409_F, price=74.4, volume=100, side=bid)}


In [39]:
NVDA_ID = "NVDA"
futures = load_futures_for_underlying(NVDA_ID)
print(futures)
ask_NVDA = exchange.get_last_price_book(NVDA_ID).asks[0].price
bid_NVDA = exchange.get_last_price_book(NVDA_ID).bids[0].price
for future in futures:
    print(calculate_theoretical_future_value(ask_NVDA, futures[future].expiry))
    print(calculate_theoretical_future_value(bid_NVDA, futures[future].expiry))

print(exchange.get_last_price_book('NVDA_202412_F'))
print(exchange.get_last_price_book('NVDA_202406_F'))
print(exchange.get_last_price_book('NVDA_202409_F'))

{'NVDA_202412_F': Instrument(instrument_id=NVDA_202412_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK_FUTURE, base_instrument_id=NVDA, expiry=2024-12-20 12:00:00, interest_rate=0.03, instrument_group=NVDA, paused=False, expired=False), 'NVDA_202406_F': Instrument(instrument_id=NVDA_202406_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK_FUTURE, base_instrument_id=NVDA, expiry=2024-06-21 12:00:00, interest_rate=0.03, instrument_group=NVDA, paused=False, expired=False), 'NVDA_202409_F': Instrument(instrument_id=NVDA_202409_F, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK_FUTURE, base_instrument_id=NVDA, expiry=2024-09-20 12:00:00, interest_rate=0.03, instrument_group=NVDA, paused=False, expired=False)}
71.247626458620